# Basics of BigDFT: N2 molecule as example

This is a simple notebook that shows how to execute a simple calculation with BigDFT.
You will learn how to manipulate basic DFT objects from a python script.
This expands some of the concepts which have been briefly introduced in the quickstart tutorial.

## Setup Environment
Recall that installing conda will crash the kernel. This is necessary.

In [ ]:
install = "client" #@param ["full_suite", "client"]
use_google_drive = False # @param {type:"boolean"}
install_var=install
!wget https://raw.githubusercontent.com/BigDFT-group/bigdft-school/data/packaging/install.py &> /dev/null
args={'locally': True} if not use_google_drive else {}
import install
getattr(install,install_var.split()[0])(**args)

Now the dependencies can be installed.

In [ ]:
install.data_archive('data/session1.tar.xz')

## How to perform a first run with default parameters

For this tutorial, we will run just a simple N2 molecule, taken from the BigDFT database (available [here](https://gitlab.com/l_sim/bigdft-suite/-/tree/devel/PyBigDFT/BigDFT/Database/XYZs)).

In [ ]:
from BigDFT.Systems import System
from BigDFT.Fragments import Fragment
from BigDFT.IO import XYZReader

N2 = System()
with XYZReader("N2") as ifile:
    N2["N:0"] = Fragment([next(ifile)])
    N2["N:1"] = Fragment([next(ifile)])

Every system object has a `posinp` representation, which is a dictionary representation of the geometry.

In [ ]:
print(N2.get_posinp())

We now run the DFT calculation (with the default input parameters) of this molecule.
This can be done by instantiating a class from the `Calculators` module. Then we call the run method of that calculator with the system of interest.

In [ ]:
from BigDFT import Calculators as C
study = C.SystemCalculator(verbose=False,skip=True) #Create a calculator
log = study.run(sys=N2,name="N2",run_dir='scratch') #run the code

The `run` method of the `BigDFT.Calculators.SystemCalculator` class shows in the standard output which is the
command that is invoked.
Then, an instance of the `Logfile` class is returned.
Such class can be used (we will see after) to extract the information about the electronic structure of the system.

Now we can retrieve important informations on the run. See the examples below:

In [ ]:
log.energy #this value is in Ha

In [ ]:
log.evals[0].tolist() # the eigenvalues in Ha ([0] stands for the first K-point, here meaningless)

In [ ]:
log.get_dos().plot() #the density of states

The tutorial [DoS Manipulation](https://githubtocolab.com/BigDFT-group/bigdft-school/blob/main/CCP_tutorials/1.J.DoS-Manipulation.ipynb) can be followed to learn how to plot Density of States from a Logfile

## How to use the SystemCalculator instance

Before managing different calculations at once, let us look closer at the `SystemCalculator` that was presented above.

In [ ]:
study = C.SystemCalculator(verbose=False,omp=1,mpi_run='mpirun -np 2', skip=True)

As you can see, initiating an instance allows to set the computational parameters such as the OpenMP and MPI parallelisation. In this case, a single thread is used while two processes are running in parallel, therefore reducing the computation time.
The global options of the runner (or calculator) can then be accessed by

In [ ]:
study.global_options()

Global options can be also added and removed as follows

In [ ]:
study.update_global_options(new_option = 'value')
study.global_options()

Similarly, global options can also be removed

In [ ]:
study.pop_global_option('new_option')
study.global_options()

Lastly, note that PyBigDFT also enables to run workflows remotely, as explained in the following tutorial.

## How to modify the input parameters from the default ones

To specify non-default input parameters to the code, we should employ a `Inputfile` object. One can choose a naming prefix for a run, which enables to classify the runs which are performed, and ease the subsequent postprocessing.

Methods of the class can be employed for modifying the input parameters. For instance, the XC functional can be chosen via the `set_xc` method.
Imagine for example that you are interested in visualizing the wavefunctions output of the calculation. To do that, the suitable method of the class instance should be called.
Create a new calculation set by using, for instance, the `LDA` prefix.

In [ ]:
from BigDFT import Inputfiles as I

inp = I.Inputfile()
inp.set_xc('LDA')
inp.write_orbitals_on_disk()

The wavefunctions will be present at the end of calculation, by indicating the value of the key *orbitals* in the output dictionary.

In [ ]:
study.update_global_options(skip=True) # this would avoid to rerun the calculation and consider the data on the drive

In [ ]:
#Run the code with the LDA prefix
LDA = study.run(name="LDA",input=inp,posinp=N2.get_posinp(),run_dir='scratch')

When using a naming scheme, the output files are placed in a directory called  **data-LDA**. In our LDA example, the wavefunctions of the system can thus be found in the **data-LDA** directory:

In [ ]:
!ls scratch/data-LDA

Here 001 means the first K-point (meaningless in this case), N stands for non spin-polarized, R for real part and the remaining number is the orbital ID. Post-processing of these files is done in the another tutorial.

In the same spirit, another calculation can be done with different parameters. Imagine we want to perform a Hartree-Fock calculation.
We should simply change the XC functional used. However, we also have to specify the pseudopotential.

In [ ]:
inp.set_xc('HF')
inp['psppar.N']={'Pseudopotential XC': 1} #this set the default PSP for LDA
HF = study.run(name="HF",input=inp,run_dir='scratch') #Run the code with the name scheme HF

Without specifying the PSP, an error will occur. The same error is specified at the end of the log file,
and also in [debug/bigdft-err-0.yaml](./scratch/debug/bigdft-err-0.yaml):
<pre>
 Additional Info:
   The pseudopotential parameter file "psppar.N" is lacking, and no registered pseudo found
   for "N"</pre>

This is because the pseudopotential is assigned by default in the code only for LDA and PBE XC approximations.
Another alternative is to specify the internal PSP that might be used, taking from the default database of BigDFT. This can be done as follows:

In [ ]:
inp['psppar.N']={'Pseudopotential XC': 1} #here 1 stands for LDA as per the XC codes

There are also some routines built into PyBigDFT for setting Krack or NLCC pseudoptentials.

In [ ]:
inp.set_psp_krack(functional="LDA")
inp.set_psp_nlcc()

When possible, care should be taken in choosing a pseudopotential which has been generated with the same XC approximation used. Unfortunately, at present HGH data are only available for semilocal functionals. For example, the same exercise as follows could have been done with Hybrid XC functionals, like for example PBE0 (ixc=-406). In the case of Hartree-Fock calculations, using semilocal functionals generally yield accurate results (see [Physical Review B 37.5 (1988): 2674](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.37.2674)).

In BigDFT, XC functionals are specified using the built in named functionals, or using the [LibXC codes](https://www.tddft.org/programs/libxc/functionals/).

In [ ]:
inp.set_xc('PBE0')
PBE0 = study.run(name="PBE0",input=inp,run_dir='scratch') #Run the code with the name scheme PBE0

The variables *LDA*, *HF*, and *PBE0* contains all information about the calculation. This is a class Logfile which simplify considerably the extraction of parameters of the associated output file *log-LDA.yaml*. If we simply type:

In [ ]:
print(LDA)

We display some information about the LDA calculation. For instance, we can extract the eigenvalues of the Hamiltonian *i.e.* the DOS (Density of States):

In [ ]:
LDA.evals[0].tolist()

Note that *LDA.log* is the python dictionary associated to the full output file.
This [tutorial](./CCP_tutorials/1.I.Logfile-basics.ipynb) explains the basic methods.

In [ ]:
print(PBE0)

The following exercise (and its [solution](../CCP_tutorials/1.E.N2-solution.ipynb)) gives some clues about it.

## The overall structure of files in the disk

In the disk, after that the run is performed, different files are created:
* `input_minimal.yaml` which contains the minimal set of input variables to perform the run;
* `log.yaml` which contains the log of the run with all calculated quantities;
* `time.yaml` is associated to the time profiling.

For its I/O, BigDFT uses the [yaml](http://www.yaml.org/spec) format.
If you look at the `N2_minimal.yaml` file, you can see:

In [ ]:
! cat scratch/N2_minimal.yaml

In this case, only the atomic positions are indicated using a yaml format.

However, there is a one-to-one correspondence between a yaml file and a python dictionary. For this reason
we will create the input parameters of this runs from dictionaries. See the useful [yaml online parser](http://yaml-online-parser.appspot.com/) page to understand the correspondence.

In the log file `log-N2.yaml`, BigDFT automatically displays all the input parameters used for the calculations.

The parameters which were not explicitly given are set to a default value, except the atomic positions of course, which have to be given. As we did not specified input files here, our run is a single-point LDA calculation, without spin-polarisation.